In [21]:
#import required Library
import random
import requests
import mysql.connector
import json

#create the list to Generate the 15 unique random number.
randomlist = random.sample(range(1, 87), 15)
print(randomlist)

[23, 28, 84, 29, 9, 32, 72, 66, 5, 27, 10, 70, 35, 16, 46]


In [22]:
#create class to add data in dictonary
class create_dict(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value

In [23]:
#MySql - Making Connection
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="pass@123"
)

#create cursor object
mycursor = mydb.cursor()

#execute DDL query (either we can call from here or we can remove it from here and run from MYSQL)
mycursor.execute("CREATE DATABASE if not exists comic_db")
mycursor.execute("CREATE TABLE if not exists comic_db.comic (comic_name VARCHAR(255), alt_text VARCHAR(255), number int, link VARCHAR(255),image VARCHAR(255), imageLink VARCHAR(255))")
mycursor.execute("TRUNCATE TABLE comic_db.comic")

In [25]:
recordList=[]
#get the random unique number from list
for x in randomlist:
    #get comic information from XKCD API
    getComic = requests.get("https://xkcd.com/{0}/info.0.json".format(x))
    
    #check the response code
    if (getComic.status_code == 200):
        #get response json
        comic_details=getComic.json()
        
        #fetch column details from json
        comicName=comic_details["title"]
        comicAltText=comic_details["alt"]
        comicNumber=comic_details["num"]
        comicLink=comic_details["link"]
        comicImg=comic_details["img"].split("/")[-1]
        comicImgLink=comic_details["img"]
        
        #append in list for bulk insert
        recordList.append((comicName,comicAltText,comicNumber,comicLink,comicImg,comicImgLink))
    elif (response.status_code == 404):
        print("Result not found!")
        exit

#bulk insert into MYSQL db
insert_sql = "INSERT INTO comic_db.comic (comic_name, alt_text, number, link, image, imageLink) VALUES (%s, %s, %s, %s, %s, %s)"
mycursor.executemany(insert_sql, recordList)

#commit insert data
mydb.commit()
print(mycursor.rowcount, "record inserted.")

15 record inserted.


In [26]:
#select data using query
mycursor.execute("SELECT * FROM comic_db.comic")
myresult = mycursor.fetchall()

final=[]
#loop for showing the data on console
for row in myresult:
    mydict = create_dict()
    mydict.add("comic",row[0])
    mydict.add("comic_meta",({"alt_text":row[1],"number":row[2],"link":row[3],"image":row[3],"image_link":row[3]}))
    final.append(mydict)
    
stud_json = json.dumps(final, indent=2, sort_keys=True)
print(stud_json)

#terminate connection
mycursor.close()

[
  {
    "comic": "T-shirts",
    "comic_meta": {
      "alt_text": "It's depressing how many of these are real shirts",
      "image": "",
      "image_link": "",
      "link": "",
      "number": 23
    }
  },
  {
    "comic": "Elefino",
    "comic_meta": {
      "alt_text": "Hell if I know",
      "image": "",
      "image_link": "",
      "link": "",
      "number": 28
    }
  },
  {
    "comic": "National Language",
    "comic_meta": {
      "alt_text": "She's pretty sharp when provoked.",
      "image": "",
      "image_link": "",
      "link": "",
      "number": 84
    }
  },
  {
    "comic": "Hitler",
    "comic_meta": {
      "alt_text": "So he's saying that God thought Hitler's art was so bad that the Holocaust was an acceptable alternative.  It's no secret that the hat guy is closely based on Aram, from Men in Hats.",
      "image": "",
      "image_link": "",
      "link": "",
      "number": 29
    }
  },
  {
    "comic": "Serenity is coming out tomorrow",
    "comic_met

True